# Importando as bibliotecas

In [ ]:
import os
import ee
import geemap

# Definindo a área de interesse

In [ ]:
# Defina no mapa a sua área de interesse utilizando as ferramentas de desenho (ponto, círculo, retângulo ou polígono)
Map = geemap.Map()
Map

In [ ]:
area_int = Map.user_roi
area_int.getInfo()

# Definindo a função

In [ ]:
# Você pode alterar para outras coleções de imagens, do Landsat ou não, alterando o ID dentro do "ee.ImageCollection()".
# Caso altere para outra coleção de imagens que não o Landsat é necessário verificar qual o nome do metadado de cobertura
# da nuvem da coleção. Substituir o nome "CLOUD_COVER" dentro do filtro .sort()

def melhores_imagens(ano):
    ano_inicio = ee.Date.fromYMD(ano, 1, 1)
    ano_fim = ano_inicio.advance(1, 'year')
    
    imagem = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
                .filterDate(ano_inicio, ano_fim) \
                .filterBounds(area_int) \
                .sort("CLOUD_COVER") \
                .first() \
                
    return imagem

In [ ]:
# Definir o período que você quer analisar, substituindo o ano incial e o ano final dentro da função ee.List.sequence()

anos = ee.List.sequence(1984, 2011)

In [ ]:
imagens = anos.map(melhores_imagens)

# Pré-visualização

Essa etapa não é necessária para gerar o gif final, ela serve apenas para verificar algumas coisas: se a função está funcionando corretamente; se as imagens foram armazenadas corretamente na variável "imagens" após o looping; se os parâmetros de visualização estão de acordo com o seu objetivo, podendo ser alterados até se obter os parâmetros ideias.

Caso queira, pode pular essa etapa e continuar na etapa "Gerando o gif"

In [ ]:
contar = imagens.size().getInfo()
print(contar)

In [ ]:
lista_anos = anos.getInfo()
lista_anos

In [ ]:
# Aqui você pode testar os parâmetros de visualização, alterando as bandas, valores mínimos e máximos dos pixels, gamma.

p_visual = {
    'bands': ['B3', 'B2', 'B1'],
    'min': 300, 
    'max': 1000,
    'gamma': 1.0
}

In [ ]:
for index in range(0, contar):
    imagem = ee.Image(imagens.get(index))
    nome_camada = "Imagem " + str(lista_anos[index])
    Map.addLayer(imagem, p_visual, nome_camada, False)
Map

# Gerando o gif

In [ ]:
# Substituir o diretório "/mnt/d/Downloads/linux/saidas" pelo diretório onde deseja salvar o arquivo de saída.
# Mudar o nome do arquivo "palmas.gif" caso queira, lembre apenas que o arquivo deve conter .gif no final do nome.

saida_gif = os.path.join("/mnt/d/Downloads/linux/saidas", "palmas.gif")

In [ ]:
# Aqui você também pode alterar os parâmetros, tanto da parte do vídeo: dimensão e frames por segundo, como da visualização:
# bandas, valores mínimos e máximos dos pixels, gamma.

video_args = {
    'dimensions': 768,
    'region': area_int,
    'framesPerSecond': 2,
    'bands': ['B3', 'B2', 'B1'],
    'min': 300, 
    'max': 1000,
    'gamma': 1.0
}

In [ ]:
geemap.download_ee_video(ee.ImageCollection(imagens), video_args, saida_gif)

In [ ]:
# Visualizando o gif dentro do Jupyter Notebook

geemap.show_image(saida_gif)

In [ ]:
# Substituir o diretório "/mnt/d/Downloads/linux/saidas" pelo diretório onde deseja salvar o novo arquivo de saída.
# Mudar o nome do arquivo "palmas_final.gif" caso queira.
# Alterar, caso queira, parâmetros de localização do texto, sequência dos anos, tamanho da letra, cor da letra, e duração (frames por segundo).

texto_gif = os.path.join("/mnt/d/Downloads/linux/saidas", "palmas_final.gif")
geemap.add_text_to_gif(saida_gif, texto_gif, xy=('3%', '5%'), text_sequence=1984, font_size=30, font_color='#ffffff', duration=200)

In [ ]:
geemap.show_image(texto_gif)

In [ ]:
# Alterar o nome "Palmas" para a sua região de interesse analisada.

nome = "Palmas"
geemap.add_text_to_gif(texto_gif, texto_gif, xy=('2%', '88%'), text_sequence=nome, font_size=45, font_color='#ffffff', duration=200)

In [ ]:
geemap.show_image(texto_gif)